In [3]:
import holidays

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pygam import LinearGAM, te, s, l, f
import plotly.graph_objects as go

In [4]:
train_df = pd.read_csv('../data/train.csv')
train_df['date'] = pd.to_datetime(train_df['date'], utc=True)
train_df.set_index('date', inplace=True)

test_df = pd.read_csv('../data/test.csv')
test_df['date'] = pd.to_datetime(test_df['date'], utc=True)
test_df.set_index('date', inplace=True)

weather_df = pd.read_parquet('../data/meteo.parquet')
weather_df['date'] = pd.to_datetime(weather_df['date'], utc=True)
weather_df.set_index('date', inplace=True)

co_emissions_df = pd.read_csv('../data/annual-co-emissions.csv')
co_emissions_df['Year'] = pd.to_datetime(co_emissions_df['Year'].astype(str) + '-07-01', utc=True)
co_emissions_df.set_index('Year', inplace=True)

In [5]:
def load_holiday_df(df):
    fr_holidays = holidays.FR(years=range(df.index.min().year, df.index.max().year + 1))
    holiday_series = pd.Series({date: fr_holidays.get(date, 'No Holiday') for date in df.index}, index=df.index, name='holiday')
    holiday_df = pd.get_dummies(holiday_series, prefix='holiday')
    holiday_df.index = df.index 
    
    return holiday_df

In [6]:
weather_cols = ['t', 'ff', 'tminsol', 'pmer']

def load_weather_df(wdf, df):
    weather_df = wdf.copy()
    
    for col in weather_cols:
        weather_df[col] = pd.to_numeric(weather_df[col], errors='coerce')
    
    weather_df = weather_df.pivot_table(
        index='date',
        columns='numer_sta',
        values=weather_cols
    )
    
    weather_df.dropna(thresh=int(0.95 * len(weather_df)), axis=1, inplace=True)
    weather_df = weather_df.resample('30min').interpolate(method='cubic', limit_direction='both')
    weather_df = weather_df.reindex(df.index)
    
    return weather_df

In [7]:
def load_fr_co_emissions_df(codf, df):
    fr_co_emissions_df = codf[codf['Entity'] == 'France'].copy()
    fr_co_emissions_df = fr_co_emissions_df.drop(columns='Entity')
    fr_co_emissions_df = fr_co_emissions_df.resample('30min').interpolate(method='cubic', limit_direction='both')
    fr_co_emissions_df = fr_co_emissions_df.reindex(df.index)
    
    return fr_co_emissions_df


In [8]:
X_weather_df = load_weather_df(weather_df, train_df)
X_holiday_df = load_holiday_df(train_df)
X_fr_co_emissions_df = load_fr_co_emissions_df(co_emissions_df, train_df)

In [9]:
plt.figure(figsize=(12, 10))
sns.heatmap(train_df.isnull(), cbar=False, cmap='viridis')

years = train_df.index.year
unique_years = np.sort(np.unique(years))

tick_positions = [years.tolist().index(y) for y in unique_years if y in years.tolist()]

plt.yticks(ticks=tick_positions, labels=unique_years)
plt.xlabel('Locations', fontsize=14) 
plt.ylabel('Year', fontsize=14) 
plt.title('Missing Values in Load Data', fontsize=16) 

plt.tight_layout(pad=2.0) 
plt.savefig("figures/nan.png", dpi=300)
plt.close()

In [10]:
fr_load = train_df['France'][:48*7]

fig = go.Figure(go.Scatter(x=fr_load.index, y=fr_load.values.ravel(), mode='lines', name='France load'))

fig.update_layout(title="France load during a week", xaxis_title='Date', yaxis_title='France Load')
fig.write_image("figures/load_wk.png", scale=2)


In [11]:
fr_load = train_df['France']
fr_load_smooth = train_df['France'].rolling(window=48*30, center=True).mean()

fig = go.Figure(go.Scatter(x=fr_load.index, y=fr_load.values.ravel(), mode='lines', name='France load'))
fig.add_trace(go.Scatter(x=fr_load_smooth.index, y=fr_load_smooth.values.ravel(), mode='lines', name='Smoothed France load'))

lockdown_start = pd.to_datetime('2020-03-17')
lockdown_end = pd.to_datetime('2020-05-11')

fig.add_shape(type="line", x0=lockdown_start, x1=lockdown_start, y0=fr_load.min(), y1=fr_load.max(), line=dict(color="orange", width=2, dash="dash"))
fig.add_shape(type="line", x0=lockdown_end, x1=lockdown_end, y0=fr_load.min(), y1=fr_load.max(), line=dict(color="orange", width=2, dash="dash"))

fig.update_layout(title="France load throughout the years", xaxis_title='Date', yaxis_title='France Load')
fig.write_image("figures/load_yr.png", scale=2)


In [12]:
weekday = train_df.index.weekday.values.astype(float).reshape(-1, 1)
fr_load = train_df['France'].values.reshape(-1, 1)

weekday_means = train_df.groupby(train_df.index.weekday)['France'].mean()
unique_weekdays = np.unique(weekday)
weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig = go.Figure()

for i, day in enumerate(weekday_names):
    fig.add_trace(go.Bar(x=[day], y=[weekday_means[i]], name=day,))

fig.update_layout(title='Mean Load Values by Weekday', xaxis_title='Weekday', yaxis_title='Mean France Load', legend_title='Weekdays', yaxis=dict(range=[min(weekday_means) - 2000, max(weekday_means) + 2000]))
fig.write_image("figures/weekday_load.png", scale=2)

In [13]:
mean_t = X_weather_df['t'].mean(axis=1).values.reshape(-1, 1)
weekday = train_df.index.weekday.values.astype(float).reshape(-1, 1)

fr_load = train_df['France'].values.reshape(-1, 1)

X = np.hstack([mean_t, weekday])
gam = LinearGAM(te(0, 1)).fit(X, fr_load)
fr_pred = gam.predict(X)
unique_weekdays = np.unique(weekday)
weekday_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

perm = np.argsort(mean_t.ravel())

fig = go.Figure(go.Scatter(x=mean_t.ravel(), y=fr_load.ravel(), mode='markers', name='Actual load', marker=dict(color='grey', size=1)))

for wd in unique_weekdays:
    mask = weekday[perm].ravel() == wd
    fig.add_trace(go.Scatter(x=mean_t[perm][mask].ravel(), y=fr_pred[perm][mask].ravel(), mode='lines', name=weekday_names[int(wd)]))

fig.update_layout(title="France load with temperature", xaxis_title='Mean Temperature', yaxis_title='France Load')
fig.write_image("figures/gam_temp.png", scale=2)

In [14]:
mean_t = X_weather_df['t'].mean(axis=1).values.reshape(-1, 1)
mean_tminsol = X_weather_df['tminsol'].mean(axis=1).values.reshape(-1, 1)
mean_ff = X_weather_df['ff'].mean(axis=1).values.reshape(-1, 1)
mean_pmer = X_weather_df['pmer'].mean(axis=1).values.reshape(-1, 1)
doy = train_df.index.dayofyear.values.reshape(-1, 1)
mod = train_df.index.astype('int64').values.reshape(-1, 1) // 1e9 % 86400
weekday = train_df.index.weekday.values.astype(np.float32).reshape(-1, 1)
holiday = np.argmax(X_holiday_df.values, axis=1).reshape(-1, 1)
fr_co_emissions = X_fr_co_emissions_df.values.reshape(-1, 1)

fr_load = train_df['France'].values.reshape(-1, 1)

X = np.hstack([weekday, mean_t, mean_tminsol, mean_ff, mean_pmer, doy, mod, holiday])

gam = LinearGAM(te(0, 1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + f(7)).fit(X, fr_load)
res = fr_load - gam.predict(X).reshape(-1, 1)

gam = LinearGAM(l(0)).fit(fr_co_emissions, res)
res_pred = gam.predict(fr_co_emissions)

perm = np.argsort(fr_co_emissions.ravel())

fig = go.Figure(go.Scatter(x=fr_co_emissions.ravel(), y=res.ravel(), mode='markers', name='Actual residuals', marker=dict(color='grey', size=1)))
fig.add_trace(go.Scatter(x=fr_co_emissions[perm].ravel(), y=res_pred[perm].ravel(), mode='lines', name='Predicted residuals'))

fig.update_layout(title="Residuals with CO2 emissions", xaxis_title='CO2 Emissions', yaxis_title='Residuals')
fig.write_image("figures/gam_emissions.png", scale=2)

In [15]:
mean_t = X_weather_df['t'].mean(axis=1).values.reshape(-1, 1)
mean_tminsol = X_weather_df['tminsol'].mean(axis=1).values.reshape(-1, 1)
mean_ff = X_weather_df['ff'].mean(axis=1).values.reshape(-1, 1)
mean_pmer = X_weather_df['pmer'].mean(axis=1).values.reshape(-1, 1)
doy = train_df.index.dayofyear.values.reshape(-1, 1)
mod = train_df.index.astype('int64').values.reshape(-1, 1) // 1e9 % 86400
weekday = train_df.index.weekday.values.astype(np.float32).reshape(-1, 1)
holiday = np.argmax(X_holiday_df.values, axis=1).reshape(-1, 1)
fr_co_emissions = X_fr_co_emissions_df.values.reshape(-1, 1)

fr_load = train_df['France'].values.reshape(-1, 1)

X = np.hstack([weekday, mean_t, mean_tminsol, mean_ff, mean_pmer, doy, mod, holiday, fr_co_emissions])
gam = LinearGAM(te(0, 1) + s(2) + s(3) + s(4) + s(5) + s(6) + f(7) + l(8)).fit(X, fr_load)

print(gam.summary())

LinearGAM                                                                                                 
=============================================== ==========================================================
Distribution:                        NormalDist Effective DoF:                                    142.3072
Link Function:                     IdentityLink Log Likelihood:                              -1436449.0606
Number of Samples:                        85571 AIC:                                          2873184.7356
                                                AICc:                                         2873185.2198
                                                GCV:                                          7808261.7735
                                                Scale:                                        7784891.5539
                                                Pseudo R-Squared:                                   0.9414
Feature Function                  Lam

C:\Users\flheight\AppData\Local\Temp\ipykernel_9788\1362870797.py:16: UserWarning:

KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 


